In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as grs
import seaborn as sns
from IPython.display import Image
import plotly.express as px # ploty
import plotly.graph_objects as go


#한글폰트설정
plt.rc('font', family='NanumBarunGothic')
pd.set_option('display.max_rows', 100)


#모델
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMClassifier, plot_importance
import lightgbm 


from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error


pd.options.display.float_format = '{:.2f}'.format


import warnings
warnings.filterwarnings(action='ignore')

import gc, sys
gc.enable() # 자동 가비지 수거 활성화

In [4]:
# 한글 폰트 확인
import matplotlib.font_manager as fm
[(f.name, f.fname) for f in fm.fontManager.ttflist if 'NanumBarun' in f.name]

[('NanumBarunGothic',
  '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf'),
 ('NanumBarunGothic', '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf')]

## 데이터 불러오기

In [5]:
base_path = '/content/drive/MyDrive/이어드림스쿨/Project/Final_project/2021 농산물 예측대회 AI 경진대회/2021 농산물 가격예측 AI 경진대회/프로젝트 data/'

In [6]:
df = pd.read_parquet(base_path + 'new_total_df2.parquet')
df.head()

,SALEDATE,WHSAL_NM,CMP_NM,PUM_NM,KIND_NM,DAN_NM,POJ_NM,SIZE_NM,LV_NM,SAN_NM,DANQ,QTY,COST,TOT_QTY,TOT_AMT
0,2016-01-01,충주도매시장,중원청과,사과,후지,kg,상자,.,상,충북 충주시,15.00,1.00,26000.00,15.00,26000.00
1,2016-01-01,충주도매시장,중원청과,사과,후지,kg,상자,.,상,충북 충주시,20.00,2.00,10000.00,40.00,20000.00
2,2016-01-01,충주도매시장,중원청과,사과,후지,kg,상자,.,상,충북 충주시,14.00,1.00,10500.00,14.00,10500.00
3,2016-01-01,충주도매시장,중원청과,사과,후지,kg,상자,.,상,충북 충주시,20.00,12.00,17500.00,240.00,210000.00
4,2016-01-01,충주도매시장,중원청과,사과,후지,kg,상자,.,상,충북 충주시,20.00,7.00,25000.00,140.00,175000.00


## 사과 품종으로 test

In [ ]:
df_apple = df[df.PUM_NM == '사과']
df_apple=df_apple[['SALEDATE', 'PUM_NM', 'KIND_NM', 'SAN_NM', 'TOT_AMT','TOT_QTY']]
df_apple.head()

,SALEDATE,PUM_NM,KIND_NM,SAN_NM,TOT_AMT,TOT_QTY
0,2016-01-01,사과,후지,충북 충주시,26000.00,15.00
1,2016-01-01,사과,후지,충북 충주시,20000.00,40.00
2,2016-01-01,사과,후지,충북 충주시,10500.00,14.00
3,2016-01-01,사과,후지,충북 충주시,210000.00,240.00
4,2016-01-01,사과,후지,충북 충주시,175000.00,140.00


In [ ]:
df_apple=df_apple.groupby(['SALEDATE']).mean()
df_apple['mean_price']=df_apple['TOT_AMT']/df_apple['TOT_QTY']
df_apple=df_apple.drop(columns=['TOT_AMT','TOT_QTY'])
df_apple=df_apple.reset_index()
df_apple = df_apple.round()
df_apple['year'] = df_apple['SALEDATE'].dt.year
df_apple['month'] = df_apple['SALEDATE'].dt.month
df_apple['day'] = df_apple['SALEDATE'].dt.day
df_apple['weekday'] = df_apple['SALEDATE'].dt.weekday
df_apple.head()

,SALEDATE,mean_price,year,month,day,weekday
0,2016-01-01,1267.00,2016,1,1,4
1,2016-01-02,2594.00,2016,1,2,5
2,2016-01-04,1860.00,2016,1,4,0
3,2016-01-05,1676.00,2016,1,5,1
4,2016-01-06,1688.00,2016,1,6,2


# 전품종 modeling (함수화)

In [7]:
# 함수화

def split_train_and_test(df, date, week):
    """
    Dataframe에서 train_df, test_df로 나눠주는 함수
    df : 시계열 데이터 프레임
    date : 기준점 날짜
    """
    train = df[df['SALEDATE'] < date]
    test = df[df['SALEDATE'] >= date]
    del train['SALEDATE']
    del test['SALEDATE']
    y_train = train.pop(f'{week}week')
    x_train=train.copy()
    y_test = test.pop(f'{week}week')
    x_test=test.copy()
    return x_train,y_train,x_test,y_test

def eval_model(y_test,pred):
    y_true, y_pred = np.array(y_test), np.array(pred)
    mae = mean_absolute_error(y_true, pred)
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print('mae:',mae,'mape:',mape, 'mse:',mse)
    return mae, mape, mse

#시각화 그래프 
def eval_visul(y_test,pred,title,week):
    y_test=y_test.reset_index()[f'{week}week']
    plt.figure(figsize=(20,5))
    plt.title(title, fontsize = 25)
    plt.plot(y_test, label='true')
    plt.plot(pred, label='pred')
    plt.legend()
    plt.show()
    eval_model(y_test,pred)

def selcet_pum(df_apple,pum):
    #7가지 품목중 1가지를 집중해서 살펴보기 위해 배추를 기준으로 데이터 추출
    df_apple=df[df['PUM_NM']==pum]
    #기본만
    df_apple=df_apple[['SALEDATE', 'PUM_NM', 'KIND_NM', 'SAN_NM', 'TOT_AMT',
        'TOT_QTY']]
   #  df_apple=df_apple[['SALEDATE', 'PUM_NM', 'KIND_NM', 'SAN_NM',  'TOT_AMT',
   #     'TOT_QTY', 'year', 'month', 'day', 'weekday', 'domestic_mean',
   #     'abroad_mean', 'output', 'area', '소비자물가지수']]
    return df_apple

def mean_price(df_apple):
    #날짜기준으로 데이터 평균 계산
    df_apple=df_apple.groupby(['SALEDATE']).mean()
    #kg당 가격 산출
    df_apple['mean_price']=df_apple['TOT_AMT']/df_apple['TOT_QTY']
    #사용된 총금액과 총거래량은 제거
    df_apple=df_apple.drop(columns=['TOT_AMT','TOT_QTY'])
    df_apple=df_apple.reset_index()
    df_apple = df_apple.round()
    #날짜 컬럼 추가
    df_apple['year'] = df_apple['SALEDATE'].dt.year
    df_apple['month'] = df_apple['SALEDATE'].dt.month
    df_apple['day'] = df_apple['SALEDATE'].dt.day
    df_apple['weekday'] = df_apple['SALEDATE'].dt.weekday
    return df_apple

#n주일 후 가격을 예측하는 컬럼을 추가 
#df-> 예측기간 가격이 0으로 나오는 값 제외, 따로 변수로 지정
def forcast_week(df,week):
    df[f'{week}week']=0
    
    for index in range(len(df)):
        try:df[f'{week}week'][index] = df['mean_price'][index+7*week]
        except:continue
    df_week=df.iloc[-7*week:]
    df = df.drop(df.iloc[-7*week:].index)
    return df, df_week


## time lag는 보류

# #최대 28일 전까지 가격 데이터를 컬럼에 추가
# #시계열 데이터의 분포를 학습시키기 위해
# def train_serise(df_apple):
#     for lag in range(1,29):
#         df_apple[f'p_lag_{lag}'] = -1
#         #df_apple[f'q_lag_{lag}'] = -1
#         for index in range(lag, len(df_apple)):
#             df_apple.loc[index, f'p_lag_{lag}'] = df_apple['mean_price'][index-lag] #1일전, 2일전, ... 가격을 feature로 추가
#             #df_apple.loc[index, f'q_lag_{lag}'] = df_apple['TOT_QTY'][index-lag] #1일전, 2일전, ... 거래량을 feature로 추가
#     return df_apple


In [8]:
# 함수화

def baseline_modeling(pum) : 
    df_pum=selcet_pum(df, pum)
    display(pum, df_pum.head(2))
    df_pum=mean_price(df_pum)
    #몇주를 예상할지
    #def forcast_week(df,week):
    df_pum,df_1week=forcast_week(df_pum,1)

    display(df_pum.head(2))
    # df_pum=train_serise(df_pum) # 보류
    # 데이터프레임 테스트를 위해 나눌 기준일, 몇주 예상
    #def split_train_and_test(df, date,week):
    X_train,y_train,X_test,y_test=split_train_and_test(df_pum,'2019-09-22',1)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    # x_columns = list(df_pum.columns)
    # x_columns.remove('SALEDATE')
    # x_columns.remove('mean_price')
    # x_columns

    # scaler = StandardScaler()

    # X_train = scaler.fit_transform(X_train)
    # X_train = pd.DataFrame(X_train, index= range(X_train.shape[0]), columns = x_columns) # feature importances 그래프를 위해 dataframe화
    # X_test = scaler.transform(X_test) 
    # X_test = pd.DataFrame(X_test, index= range(X_test.shape[0]), columns = x_columns)


    # LinearRegression

    model_lr = LinearRegression().fit(X_train, y_train)
    pred = model_lr.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    mape = mean_absolute_percentage_error(pred, y_test)
    mse = mean_squared_error(pred, y_test)
    print('LinearRegression mae : %f' %mae, '/ mape : %f' %mape, '/ mse : %f' %mse)


    # Lasso

    model_ls = Lasso().fit(X_train, y_train)
    pred2 = model_ls.predict(X_test)
    mae = mean_absolute_error(pred2, y_test)
    mape = mean_absolute_percentage_error(pred2, y_test)
    mse = mean_squared_error(pred2, y_test)
    print('Lasso mae : %f' %mae, '/ mape : %f' %mape, '/ mse : %f' %mse)


    # Ridge

    model_rg = Ridge().fit(X_train, y_train)
    pred3 = model_rg.predict(X_test)
    mae = mean_absolute_error(pred3, y_test)
    mape = mean_absolute_percentage_error(pred3, y_test)
    mse = mean_squared_error(pred3, y_test)
    print('Ridge mae : %f' %mae, '/ mape : %f' %mape, '/ mse : %f' %mse)


    # LGBMRegressor

    model_lgb = LGBMRegressor().fit(X_train, y_train)
    pred4 = model_lgb.predict(X_test)
    mae = mean_absolute_error(pred4, y_test)
    mape = mean_absolute_percentage_error(pred4, y_test)
    mse = mean_squared_error(pred4, y_test)
    print('LGBMRegressor mae : %f' %mae, '/ mape : %f' %mape, '/ mse : %f' %mse)



    # XGBRegressor

    model_xgb = XGBRegressor().fit(X_train, y_train)
    pred5 = model_xgb.predict(X_test)
    mae = mean_absolute_error(pred5, y_test)
    mape = mean_absolute_percentage_error(pred5, y_test)
    mse = mean_squared_error(pred5, y_test)
    print('XGBRegressor mae : %f' %mae, '/ mape : %f' %mape, '/ mse : %f' %mse)



    # RandomForestRegressor

    model_rfr = RandomForestRegressor().fit(X_train, y_train)
    pred6 = model_rfr.predict(X_test)
    mae = mean_absolute_error(pred6, y_test)
    mape = mean_absolute_percentage_error(pred6, y_test)
    mse = mean_squared_error(pred6, y_test)
    print('RandomForestRegressor mae : %f' %mae, '/ mape : %f' %mape, '/ mse : %f' %mse)


    # prediction vs actual
    # forcast_week=1
    eval_visul(y_test,pred,f'{pum} 품종 Linear Regression',1)
    eval_visul(y_test,pred2,f'{pum} 품종 Lasso', 1)
    eval_visul(y_test,pred3,f'{pum} 품종 Ridge', 1)
    eval_visul(y_test,pred4,f'{pum} 품종 LGB Regressor', 1)
    eval_visul(y_test,pred5,f'{pum} 품종 XGB Regressor',1)
    eval_visul(y_test,pred6,f'{pum} 품종 RandomForest Regressor',1)


    # # feature importances
    # model_lgb = pd.DataFrame(data=model_lgb.feature_importances_, index = X_train.columns, columns = ['Model feature importances'])
    # model_lgb = model_lgb.sort_values(by= 'Model feature importances', ascending = True)
    # fig = plt.figure(figsize=(10,15))
    # plt.barh(model_lgb.index, model_lgb['Model feature importances'])
    # plt.title(f'{pum} 품종 LGBMRegressor feature importances', fontsize=15)
    # plt.show()

    # model_xgb = pd.DataFrame(data=model_xgb.feature_importances_, index = x_columns, columns = ['Model feature importances'])
    # model_xgb = model_xgb.sort_values(by= 'Model feature importances', ascending = True)
    # fig = plt.figure(figsize=(10,15))
    # plt.barh(model_xgb.index, model_xgb['Model feature importances'])
    # plt.title(f'{pum} 품종 XGboost feature importances', fontsize=15)
    # plt.show()

    # model_rfr = pd.DataFrame(data=model_rfr.feature_importances_, index = x_columns, columns = ['Model feature importances'])
    # model_rfr = model_rfr.sort_values(by= 'Model feature importances', ascending = True)
    # fig = plt.figure(figsize=(10,15))
    # plt.barh(model_rfr.index, model_rfr['Model feature importances'])
    # plt.title(f'{pum} 품종 RandomForest feature importances', fontsize=15)
    # plt.show()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18407168 entries, 0 to 18407167
Data columns (total 15 columns):
 #   Column    Dtype         
---  ------    -----         
 0   SALEDATE  datetime64[ns]
 1   WHSAL_NM  category      
 2   CMP_NM    category      
 3   PUM_NM    category      
 4   KIND_NM   category      
 5   DAN_NM    category      
 6   POJ_NM    category      
 7   SIZE_NM   category      
 8   LV_NM     category      
 9   SAN_NM    category      
 10  DANQ      float64       
 11  QTY       float64       
 12  COST      float64       
 13  TOT_QTY   float64       
 14  TOT_AMT   float64       
dtypes: category(9), datetime64[ns](1), float64(5)
memory usage: 1.0 GB


In [9]:
pum_lst = df.PUM_NM.unique()
for i in pum_lst :
    baseline_modeling(i)

Output hidden; open in https://colab.research.google.com to view.